In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome import service
import time
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import csv
import pandas as pd
from re import findall as re_findall
from datetime import date

In [2]:
def driver_selenium(typ, url):
    try:
        if typ == 'Chrome':
            driver = webdriver.Chrome(executable_path='../driver/chromedriver.exe')
        elif typ == 'Firefox':
            driver = webdriver.Firefox(executable_path='../driver/geckodriver.exe')
        elif typ == 'Edge':
            driver = webdriver.Edge(executable_path='../driver/msedgedriver.exe')
        driver.get(url)
        return driver
    except:
        print('Loading driver fail')

In [3]:
def glassdoor_scrap_old(job_title, location, typ):
    driver = driver_selenium(typ, 'https://www.glassdoor.com/Job/')
    
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="sc.keyword"]').clear()
    driver.find_element(By.XPATH, '//*[@id="sc.keyword"]').send_keys(job_title)
    driver.find_element(By.XPATH, '//*[@id="sc.location"]').clear()
    driver.find_element(By.XPATH, '//*[@id="sc.location"]').send_keys(location)
    driver.find_element(By.XPATH, '//*[@id="scBar"]/div/button').click()
    
       
    time.sleep(1)
    droplist_time = driver.find_element(By.XPATH, '//*[@id="Discover"]/div/div/div[1]/div[1]/div[3]/a')#.click() # Droplist click
    driver.execute_script("arguments[0].click();", droplist_time)
    
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="JobDescriptionContainer"]/div[2]/span').click()
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="filter_fromAge"]/span[2]').click()
    time.sleep(1)
    
    # Choosing the dropdownlist to choose time
    driver.find_element(By.XPATH, '//*[@id="PrimaryDropdown"]/ul/li[5]').click() # Choose 2 week job posting
    
    # Clean appearance
    time.sleep(1)   
    driver.find_element(By.XPATH, '//*[@id="MainCol"]/div[1]/ul/li[3]').click()
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="JAModal"]/div/div[2]/span').click()    
    
    time.sleep(1)   
    driver.find_element(By.XPATH, '//*[@id="MainCol"]/div[1]/ul/li[2]').click()
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="JAModal"]/div/div[2]/span').click()
    
    return driver

In [7]:
def glassdoor_scrap(job_title, location, typ):
    driver = driver_selenium(typ, 'https://www.glassdoor.com/Job/')
    driver.implicitly_wait(10)
    
    time.sleep(1)
    driver.find_element(By.XPATH, '//*[@id="sc.keyword"]').clear()
    driver.find_element(By.XPATH, '//*[@id="sc.keyword"]').send_keys(job_title)
    driver.find_element(By.XPATH, '//*[@id="sc.location"]').clear()
    driver.find_element(By.XPATH, '//*[@id="sc.location"]').send_keys(location)
    button = driver.find_element(By.XPATH, '//*[@id="scBar"]/div/button')#.click()
    driver.execute_script("arguments[0].click();", button)
    
    # See all job click   
    time.sleep(1)
    see_all_job = driver.find_element(By.XPATH, '//*[@id="Discover"]/div/div/div[1]/div[1]/div[3]/a')#.click() # see all job click
    driver.execute_script("arguments[0].click();", see_all_job)
    time.sleep(1)
        
    # See job description detail in the right
    detail_job = driver.find_element(By.XPATH, '//*[@id="JobDescriptionContainer"]/div[2]/span')#.click() # show more detail job description
    driver.execute_script("arguments[0].click();", detail_job)
    time.sleep(1)
    
    # Droplist to choose 2 week job post
    droplist = driver.find_element(By.XPATH, '//*[@id="filter_fromAge"]/span[2]')# .click()
    driver.execute_script("arguments[0].click();", droplist)
    time.sleep(1)
    
    two_week = driver.find_element(By.XPATH, '//*[@id="PrimaryDropdown"]/ul/li[5]')#.click() # Choose 2 week job posting
    driver.execute_script("arguments[0].click();", two_week)
    time.sleep(1)   
    
    
    # Clean appearance  
    ## First close
    first_clear = driver.find_element(By.XPATH, '//*[@id="MainCol"]/div[1]/ul/li[3]')#.click()
    driver.execute_script("arguments[0].click();", first_clear)
    time.sleep(1)
    
    close_first = driver.find_element(By.XPATH, '//*[@id="JAModal"]/div/div[2]/span')#.click()    
    driver.execute_script("arguments[0].click();", close_first)
    time.sleep(1)
    
    ## Second close
    second_clear = driver.find_element(By.XPATH, '//*[@id="MainCol"]/div[1]/ul/li[2]')#.click()
    driver.execute_script("arguments[0].click();", second_clear)
    time.sleep(1)
    
    close_second =driver.find_element(By.XPATH, '//*[@id="JAModal"]/div/div[2]/span')#.click()
    driver.execute_script("arguments[0].click();", close_second)
    time.sleep(1)
    return driver

In [8]:
def load_one_page(job_title, location, typ, driver):
    # driver = glassdoor_scrap(job_title, location, typ)
    # driver.implicitly_wait(10)
    lst_job = []
    elements = driver.find_elements(By.XPATH, '//*[@id="MainCol"]/div[1]/ul/li') 
    for i in range(len(elements)):
        try:
            driver.execute_script("arguments[0].click();", elements[i])
            #time.sleep(0.2)
            one_job = elements[i]
            job_general_html = one_job.get_attribute('outerHTML')
            soup_general = BeautifulSoup(job_general_html, 'lxml')
            rating = soup_general.select('div')[0].text
            company_detail = soup_general.select('div')[1]
            job_link = soup_general.find_all('a',href=True)
            link = 'https://glassdoor.com'+job_link[0]['href']
            company_name = company_detail.select('span')[0].text
            position = company_detail.select('span')[3].text
            location = company_detail.select('span')[4].text

            text = company_detail.text
            date_post = re_findall('\d{1,}[d|h]', text)
            date_post = date_post[0]
            date_retrieved = date.today()

            # Job description
            show_detail = driver.find_element(By.XPATH, '//*[@id="JobDescriptionContainer"]/div[2]')
            driver.execute_script("arguments[0].click();", show_detail)
            job_description = driver.find_element(By.XPATH, '//*[@class="jobDescriptionContent desc"]')
            job_outer_html = job_description.get_attribute('outerHTML')
            soup = BeautifulSoup(job_outer_html, 'lxml')
            job_description_text = soup.text
            job = [position, company_name, location, rating, job_description_text, link, date_post, date_retrieved]
            lst_job.append(job)
            time.sleep(3)
        except:
            print(f'Element {i} could not clicked')
    return lst_job

In [9]:
def load_full(job_title, location, typ):
    driver = glassdoor_scrap(job_title, location, typ)
    driver.implicitly_wait(10)
    check_next_button = driver.find_element(By.XPATH, '//*[@class="nextButton css-1hq9k8 e13qs2071"]').is_enabled()
    i = 1
    lst_full = []
    while (check_next_button==True) & (i<50): # Restrict to maximal 50 pages
        lst_one_page = load_one_page(job_title, location, typ, driver)
        lst_full.append(lst_one_page)
        next_click = driver.find_element(By.XPATH, '//*[@class="nextButton css-1hq9k8 e13qs2071"]')#.click()
        driver.execute_script("arguments[0].click();", next_click)  # Javascript execute
        check_next_button = driver.find_element(By.XPATH, '//*[@class="nextButton css-1hq9k8 e13qs2071"]').is_enabled()
        i+=1
        time.sleep(2)
    return lst_full

In [10]:
job_title = 'Business Intelligence'
location = 'Germany'
typ = 'Chrome'

In [11]:
driver = glassdoor_scrap(job_title, location, typ)

C:\Users\Pham Thi Ngan\AppData\Local\Temp\ipykernel_3080\3618989861.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='../driver/chromedriver.exe')


In [10]:
lst_full = load_full(job_title, location, typ)

C:\Users\Pham Thi Ngan\AppData\Local\Temp\ipykernel_18904\3618989861.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='../driver/chromedriver.exe')


In [11]:
lst_full

[[['Business Intelligence Analyst (m/w/d) in Neckarsulm',
   'LIDL Stiftung & Co. KG',
   'Germany',
   '3.5',
   'Eine gute Idee war der Ursprung, ein erfolgreiches Konzept ist das Ergebnis. Qualität zum guten Preis möglichst vielen Menschen anbieten zu können treibt uns an - mittlerweile weltweit, denn wir sind Lidl. Hier ist das berufliche Zuhause vielfältiger Talente. Ob im Einkauf, in der Logistik, im Warengeschäft oder in der IT; ob als Impulsgeber, Gestalter oder Dienstleister der Länder. Wir suchen Anpacker, Durchstarter, Möglichmacher und bieten spannende Aufgaben und Projekte in einem dynamischen und internationalen Umfeld. Bei Lidl findet jeder seine persönliche Herausforderung. Denn Lidl lohnt sich.\n\nDeine Aufgaben\n\nSchnittstellenfunktion bei Digitalisierungs- und BI- Reporting-Themen zwischen operativen und technischen Fachbereichen als Business Intelligence Analyst (m/w/d)\nFokussieren auf SAP und/oder MicroStrategy-Projekte für unsere Fachbereiche (z.B. Einkauf, Vert

In [12]:
lst_job = []
for i in range(len(lst_full)):
    lst_job +=lst_full[i]

In [13]:
df_job = pd.DataFrame(data=lst_job, columns=['position', 'company_name', 'location', 'rating', 'job_description', 'link', 'date_post', 'date_retrived'])

In [14]:
df_job

,position,company_name,location,rating,job_description,link,date_post,date_retrived
0,Business Intelligence Analyst (m/w/d) in Necka...,LIDL Stiftung & Co. KG,Germany,3.5,"Eine gute Idee war der Ursprung, ein erfolgrei...",https://glassdoor.com/partner/jobListing.htm?p...,12d,2022-07-20
1,Inhouse Consultant Business Intelligence (m/w/d),DPD Deutschland GmbH,Hamburg,3.4,Inhouse Consultant Business Intelligence (m/w/...,https://glassdoor.com/partner/jobListing.htm?p...,13d,2022-07-20
2,Sr. Business Developer - Amazon Freight (Germa...,Amazon GmbH,Berlin,3.7,Job summary\nAmazon Freight is part of the Ama...,https://glassdoor.com/partner/jobListing.htm?p...,5d,2022-07-20
3,(Junior) Business Intelligence Analyst (m/w/d),Quentic,Berlin,3.9,Passionate people for sustainable software\n\n...,https://glassdoor.com/partner/jobListing.htm?p...,12d,2022-07-20
4,Business Analyst (m/f/d) - Shipment Routing En...,Hapag Lloyd AG,Hamburg,3.8,What is the fastest way to route a shipment of...,https://glassdoor.com/partner/jobListing.htm?p...,12d,2022-07-20
...,...,...,...,...,...,...,...,...
865,Business Intelligence Developer / Analyst (f/m...,wefox,Berlin,3.6,wefox is the world’s leading InsurTech that is...,https://glassdoor.com/partner/jobListing.htm?p...,6d,2022-07-20
866,Working Student: Business Analyst,Moody's,Berlin,3.7,"realxdata, a Moody’s Analytics company, is loo...",https://glassdoor.com/partner/jobListing.htm?p...,3d,2022-07-20
867,JUNIOR BUSINESS INTELLIGENCE DEVELOPER* SCHWER...,New Yorker,Germany,3.4,JUNIOR BUSINESS INTELLIGENCE DEVELOPER* SCHWER...,https://glassdoor.com/partner/jobListing.htm?p...,208d,2022-07-20
868,Business Intelligence Engineer,Amazon EU SARL(Germany Branch),Berlin,3.7,Experience with SQL\nExperience with data visu...,https://glassdoor.com/partner/jobListing.htm?p...,2024h,2022-07-20


In [15]:
df_job.to_csv('business_intelligence_19_07.csv', index=False)